# Модель выбытия жилищного фонда

Расчитывает выбытия жилой площади, млн кв. м. на периоде 2015 - 2035 гг. Выбытия используются в долгосрочной модели для рассчета прироста ЖФ.
ЧТО ТАКОЕ ВЫБЫТИЯ? Дом считается аварийным и выбывает, если возраст достигает срока эксплуатации. (Критерий 100% или 75%?)

In [44]:
from os import path, getcwd

import pandas as pd
import numpy as np
import sqlalchemy as sa
import re
import matplotlib.pyplot as plt

strHousesDBPath=path.join('..','DB', 'Houses.sqlite3') # путь к рабочей базе данных SQLite Houses

# константы базы данных SQLite3
strHouse_table='houses' # название таблицы в базе данных SQLite
strHouse_data_pass='houses_columns' # таблица с названиями колонок
strHouseDisps_table='disps_result' # итоговая таблица выбытий 

## Данные из БД 

<p>
    <ul>
        <li>Данные из БД загружаются в виде DataFrame </li>
        <li>Вводятся ограничения на год постройки</li>
        <li>Выбираются только данные по МКД</li>
    </ul> 
</p>


In [32]:
strSELECT_House='select * from {house_table}'.format(house_table=strHouse_table)

conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strHousesDBPath)) # connection к рабочей базе данных

pdfHouse=pd.read_sql(strSELECT_House, con=conWork)

In [33]:
wdf = pdfHouse.copy()
wdf = wdf.sort_values(by=['built_year']) # сортировка по году постройки
wdf = wdf[(wdf['built_year'] >= 1801) & (wdf['built_year'] <= 2019)] # выбор данных с 1801 по 2019
wdf.reset_index(drop=True,inplace=True)
wdf

,houseguid,built_year,house_type,is_alarm,floor_count_max,elevators_count,area_total,area_residential,foundation_type,floor_type,wall_material,heating_type,hot_water_type,cold_water_type,living_quarters_count
0,8df065e7-1bba-4a5e-b1ca-0e84b3b1359c,1801.0,None,Нет,2.0,NaN,169.0,164.9,None,None,None,None,None,None,5.0
1,0dc1ad4f-e383-40f2-9067-6c1122bc4643,1801.0,Многоквартирный дом,Нет,3.0,0.0,616.7,236.6,None,Деревянные,Кирпич,Центральное,Отсутствует,Центральное,NaN
2,ded84f7a-7994-44c9-9064-70b88b91f076,1801.0,Многоквартирный дом,Нет,4.0,0.0,550.3,231.3,Ленточный,Деревянные,Кирпич,Центральное,Закрытая с приготовлением горячей воды на ЦТП,Центральное,15.0
3,f8078873-01a8-420f-a07a-f2179620afcc,1802.0,Многоквартирный дом,Нет,5.0,0.0,8946.0,5265.0,Ленточный,Смешанные,Кирпич,Центральное,Открытая с отбором сетевой воды на горячее вод...,Центральное,0.0
4,e4c23e92-615d-4e1c-93b9-4d10e76c3ca3,1802.0,Многоквартирный дом,Нет,2.0,0.0,542.1,435.9,Ленточный,Деревянные,Кирпич,Центральное,Открытая с отбором сетевой воды на горячее вод...,Центральное,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679983,2afa1b79-645b-416f-88ba-7487c70b8273,2019.0,Многоквартирный дом,Нет,20.0,4.0,12479.5,8239.3,Свайный,Железобетонные,Железобетонная панель,Индивидуальный тепловой пункт (ИТП),Закрытая с приготовлением горячей воды на ИТП,Центральное,219.0
679984,7ca3ed9d-ccf3-440e-bbda-7a680a20433e,2019.0,None,Нет,17.0,2.0,9579.7,7241.2,None,None,None,None,None,None,153.0
679985,e9378f1a-1659-4236-86ff-0c20dc646556,2019.0,None,Нет,14.0,4.0,14226.0,8720.9,None,None,None,None,None,None,139.0
679986,d8287c8c-61b0-40c3-8997-642a72f67c87,2019.0,Многоквартирный дом,Нет,32.0,14.0,56107.4,43597.9,Сплошной,Железобетонные,Монолитные,Индивидуальный тепловой пункт (ИТП),Закрытая с приготовлением горячей воды на ЦТП,Центральное,894.0


In [34]:
wdf['house_type'].unique() # выводит типы домов, которые присутствуют в данных 

array([None, 'Многоквартирный дом', 'Специализированный жилищный фонд',
       'Жилой дом блокированной застройки',
       'Жилой дом (индивидуально-определенное здание)'], dtype=object)

In [35]:
# Оставляем только МКД и Жилые дома блокированной застройки (В данном варианте модели используются только МКД. 
# В некоторых случаях жилые дома блокированной застройки также могут классифицироваться как МКД, 
# поэтому данные по ним оставлены, хотя и не используются.)
wdf=wdf[(wdf['house_type']=='Многоквартирный дом')|
        (wdf['house_type']=='Жилой дом блокированной застройки')]

## Условия

<div class="alert alert-block alert-info">
<p>Создаются условия выбытия:
    <ul>
        <li>9 вариантов для домов различных по материалу несущих стен и, соответственно, расчитанных на различный срок эксплуатации. Причем панельные и кирпичные дома разбиваются на низкоэтажные (до 5 этажей) и высокоэтажные (6 и более) </li>
        <li>1 вариант для любого вида домов, признаных аварийным </li>
    </ul> 
Результаты будут представлены по всем типам домов. Т.к. часть кирпичных и панельных "пятиэтажек" вошла в программу реновации - эти дома выделяются в отдельные типы. 
Накладывая каждое условие по почереди на совокупность данных - будет рассчитываться площадь ЖФ, которая подлежит выбытию в каждый год. Например, срок жизни деревянного (многоквартирного) дома составляет 50 лет. Соответственно, в 2020 году из эксплуатации должны выйти дома такого типа, построенные в 1970 г. В 2021 г. - построенные в 1971 г.

    
Срок службы домов определяется следующим образом:
    <ol>
        <li>Деревянные дома – 50 лет, поскольку относятся к IV группе капитальности согласно Приказу Госстроя СССР от 08.09.1964 №147.</li>
        <li>Панельные дома (1-5 этажей)  – 60 лет. У «хрущевок» официальный срок службы составляет 50 лет (при этом для сносимых серий он был заявлен в 25 лет, но дома прослужили дольше), однако при своевременном и современном обслуживании (укрепление фундамента, ремонт коммуникаций) его можно продлить до 70 лет. Таким образом, взято среднее значение.</li>
        <li>Дома со смешанными несущими стенами – 80 лет, т.к. под «смешанными» стенами подразумеваются железобетонные и деревянные или кирпичные и деревянные. Экспертно, при расчете срока службы домов, вес деревянных стен принят в 40%, кирпичных или железобетонных – в 60%.</li>
        <li>Дома с иным типом стен – 60 лет. Срок службы данных домов экспертно приравнен к сроку службы панельных пятиэтажек, поскольку неясен тип используемых материалов. Кроме того, в основном это малоэтажные дома.</li>
        <li>Дома с блочными типами стен – 100 лет, т.к. такие дома преимущественно относятся к III группе капитальности согласно Приказу Госстроя СССР от 08.09.1964 №147.</li>
        <li>Кирпичные дома (1-5 этажей) – 100 лет, т.к. соответствует III группе капитальности согласно Приказу Госстроя СССР от 08.09.1964 №147.</li>
        <li>Кирпичные дома (6+ этажей) – 125 лет. Срок службы кирпичных домов может варьироваться от 100 до 150 лет в зависимости от толщины кирпичной кладки стен и относиться к I-III группе капитальности согласно Приказу Госстроя СССР от 08.09.1964 №147. Таким образом, взято среднее значение.</li>
        <li>Панельные дома (6+ этажей) – 100 лет, т.к. соответствуют III группе капитальности согласно Приказу Госстроя СССР от 08.09.1964 №147.</li>
        <li>Монолитные дома – 150 лет, поскольку относятся к I группе капитальности согласно Приказу Госстроя СССР от 08.09.1964 №147.</li>
    </ol>
</p>
</div>

In [36]:
# Каждое условие записывается в отдельную переменную "cond", содержащую информацию по: доле учитываемого фонда, возрасту, этажности (от и до), аварийности, типу несущих стен и типу дома.
cond1=pd.Series({'Доля учитываемого фонда': 1, 
                 'Возраст': 50,
                 'Аварийный': 'Нет', 
                 'Несущие стены': 'Деревянные', 
                 'Тип дома': 'Многоквартирный дом'}, name='Деревянные')
cond2=pd.Series({'Доля учитываемого фонда': 0.7,
                 'Возраст': 60, 
                 'Этажность (от)': 1,
                 'Этажность (до)': 5,
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Панельные', 
                 'Тип дома': 'Многоквартирный дом'}, name='Панельные (1-5 этажей)')
cond3=pd.Series({'Доля учитываемого фонда': 0.5,
                 'Возраст': 80, 
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Смешанные', 
                 'Тип дома': 'Многоквартирный дом'}, name='Смешанные')
cond4=pd.Series({'Доля учитываемого фонда': 0.5,
                 'Возраст': 60,
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Иные', 
                 'Тип дома': 'Многоквартирный дом'}, name='Иные')
cond5=pd.Series({'Доля учитываемого фонда': 0.3,
                 'Возраст': 100,
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Блочные', 
                 'Тип дома': 'Многоквартирный дом'}, name='Блочные')
cond6=pd.Series({'Доля учитываемого фонда': 0.3,
                 'Возраст': 100, 
                 'Этажность (от)': 1,
                 'Этажность (до)': 5,
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Кирпич', 
                 'Тип дома': 'Многоквартирный дом'}, name='Кирпичные (1-5 этажей)')
cond7=pd.Series({'Доля учитываемого фонда': 0.1,
                 'Возраст': 100, 
                 'Этажность (от)': 6,
                 'Этажность (до)': 50,
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Кирпич', 
                 'Тип дома': 'Многоквартирный дом'}, name='Кирпичные (6+ этажей)')
cond8=pd.Series({'Доля учитываемого фонда': 0.5,
                 'Возраст': 100, 
                 'Этажность (от)': 6,
                 'Этажность (до)': 50,
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Панельные', 
                 'Тип дома': 'Многоквартирный дом'}, name='Панельные (6+ этажей)')
cond9=pd.Series({'Доля учитываемого фонда': 0.1,
                 'Возраст': 150,
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Монолитные', 
                 'Тип дома': 'Многоквартирный дом'}, name='Монолитные')
cond10=pd.Series({'Аварийный': 'Да',
                 'Тип дома': 'Многоквартирный дом'}, name='Аварийные')

Cond_df=pd.DataFrame([cond1,cond2,cond3,cond4,cond5,cond6,cond7,cond8,cond9,cond10])
print('Итоговая таблица условий выглядит следующим образом:')
Cond_df

Итоговая таблица условий выглядит следующим образом:


,Доля учитываемого фонда,Возраст,Аварийный,Несущие стены,Тип дома,Этажность (от),Этажность (до)
Деревянные,1.0,50.0,Нет,Деревянные,Многоквартирный дом,NaN,NaN
Панельные (1-5 этажей),0.7,60.0,Нет,Панельные,Многоквартирный дом,1.0,5.0
Смешанные,0.5,80.0,Нет,Смешанные,Многоквартирный дом,NaN,NaN
Иные,0.5,60.0,Нет,Иные,Многоквартирный дом,NaN,NaN
Блочные,0.3,100.0,Нет,Блочные,Многоквартирный дом,NaN,NaN
Кирпичные (1-5 этажей),0.3,100.0,Нет,Кирпич,Многоквартирный дом,1.0,5.0
Кирпичные (6+ этажей),0.1,100.0,Нет,Кирпич,Многоквартирный дом,6.0,50.0
Панельные (6+ этажей),0.5,100.0,Нет,Панельные,Многоквартирный дом,6.0,50.0
Монолитные,0.1,150.0,Нет,Монолитные,Многоквартирный дом,NaN,NaN
Аварийные,NaN,NaN,Да,NaN,Многоквартирный дом,NaN,NaN


In [37]:
# Переименовываем имена колонок таблицы условий в совпадающие с именами в БД
Cond_df.rename(columns={'Доля учитываемого фонда': 'share',
                        'Возраст':'age',
                        'Аварийный':'is_alarm',
                        'Несущие стены':'wall_material', 
                        'Тип дома':'house_type',
                        'Этажность (от)':'floor_0', 
                        'Этажность (до)':'floor_1' }, inplace = True)

Поскольку значение суммарной жилой площади ЖФ расчитанное по БД оказывается меньше того, что публикует росстат, 
<br>полученные по панели для конкретного условия результаты выбытия корректируются на коэффициент:
<br>(Жилая площадь, по состоянию на конец 2019 г.)/(суммарная жилая площадь в панели данных по МКД конкретного вида)
<br>При расчете суммарной жилой площади используются только данные без пропусков каждому из параметров условий.
<br>Жилая площадь, по состоянию на конец 2019 г. (млн кв. м.) (источник - https://fedstat.ru/indicator/57479):

In [38]:
HouseSqTot=2353.040248

## Расчитываем выбытия площадей ЖФ по заданным условиям
Результаты выбытий представлены накопленным итогом (т.е. объем жилплощади, которая должна быть выведена из эксплуатации к данному году)

In [40]:
# Создаем пустой датафрейм для вывода подсчетов выбытий жилплощадей ЖФ; заполняем его, используя по очереди каждое условие. 
# Информация о рассчитываемом в данный момент условии и успешном его завершении будет появляться ниже, под кодом.
w_cols=list(range(2015, 2036)) # задается период 2015 - 2035 гг.
#idx=['cond'+str(i) for i in range(1,11)] 
fdf=pd.DataFrame(columns=w_cols, index=Cond_df.index)

for i, row in Cond_df.iterrows():
    d_main=row.to_dict()
    
    print(i,end='...')
    
    # Все, кроме аварийных
    if pd.notna(d_main['age']):
        age=d_main['age']
        share=d_main['share']
        #print('age:',age,'share:',share)
        
        # Для условий, в которых нет ограничений по количеству этажей
        if pd.isna(d_main['floor_0']):
            d_body={k: d_main[k] for k in d_main if (pd.notna(d_main[k])) and k not in ['age', 'share']}
            query_age="(col-wdf['built_year']>={})".format(age)
            query_body = ' & '.join(['(wdf[{}]=={})'.format(repr(k),repr(v)) for k,v in d_body.items()])
            query=query_body+' & '+query_age
            #print(query)
            
        # Для условий, в которых есть ограничения по количеству этажей
        else:
            d_body={k: d_main[k] for k in d_main if pd.notna(d_main[k]) and k not in ['age','share','floor_0','floor_1']}
            d_floor={k: d_main[k] for k in ['floor_0','floor_1']}
            query_age="(col-wdf['built_year']>={})".format(age)
            query_body = ' & '.join(['(wdf[{}]=={})'.format(repr(k),repr(v)) for k,v in d_body.items()])
            query_floor = "(wdf['floor_count_max'].between({},{}))".format(d_floor['floor_0'],d_floor['floor_1'])
            query=query_body+' & '+query_floor+' & '+query_age
            d_body.update({'floor_count_max':'no matter'})
            #print(query)
    
    # Аварийные
    else:
        share=1
        d_body={k: d_main[k] for k in d_main if pd.notna(d_main[k])}
        query = ' & '.join(['(wdf[{}]=={})'.format(repr(k),repr(v)) for k,v in d_body.items()])
        #print(query)

    for col in fdf.columns:
        fdf.loc[i,col] = (wdf[eval(query)]['area_residential'].sum()/1000000) * HouseSqTot / (wdf.dropna(subset=d_body.keys())['area_residential'].sum()/1000000) * share
    print('условие посчитано')    
    
fdf.loc['Total']=fdf.sum()

print('Итоговая таблица выбытий (суммарные выбытия в строке Total):')
fdf

Деревянные...условие посчитано
Панельные (1-5 этажей)...условие посчитано
Смешанные...условие посчитано
Иные...условие посчитано
Блочные...условие посчитано
Кирпичные (1-5 этажей)...условие посчитано
Кирпичные (6+ этажей)...условие посчитано
Панельные (6+ этажей)...условие посчитано
Монолитные...условие посчитано
Аварийные...условие посчитано
Итоговая таблица выбытий (суммарные выбытия в строке Total):


,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,...,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
Деревянные,9.40654,9.64505,9.8934,10.1703,10.43,10.7171,10.9381,11.157,11.3748,11.5944,...,12.0348,12.237,12.4709,12.707,13.0023,13.2558,13.5506,13.896,14.2024,14.556
Панельные (1-5 этажей),0.468901,0.550223,0.718832,1.06393,1.62073,3.04378,5.78152,10.1273,15.514,21.8578,...,37.3896,45.7771,54.6586,63.1303,72.1257,80.8443,90.2108,99.7086,109.176,118.281
Смешанные,0.593903,0.607049,0.650951,0.660655,0.675848,0.690231,0.69703,0.699013,0.700021,0.702659,...,0.747893,0.765004,0.807892,0.861907,0.959818,1.03399,1.18314,1.30162,1.43271,1.55005
Иные,1.19217,1.35659,1.54001,1.80691,2.11329,2.43865,2.7793,3.18841,3.61826,3.96451,...,4.55857,4.95967,5.60502,6.28313,6.8476,7.40069,8.11312,8.7648,9.32675,9.9053
Блочные,0.0239144,0.0239144,0.0376328,0.038071,0.0385676,0.0389888,0.0390242,0.0390242,0.0392545,0.0393173,...,0.0437999,0.0452462,0.0483859,0.05129,0.0568433,0.0668193,0.0762747,0.0833256,0.0924972,0.10406
Кирпичные (1-5 этажей),3.40391,3.57113,4.78309,4.82442,4.82928,4.83875,4.8404,4.8484,4.85873,4.87299,...,4.99144,5.11129,5.29055,5.52044,5.89612,6.14707,6.45905,6.65988,6.90871,7.14301
Кирпичные (6+ этажей),0.603601,0.624632,0.680562,0.680562,0.680562,0.681071,0.681071,0.681071,0.682051,0.686531,...,0.706342,0.728411,0.757605,0.78062,0.793052,0.808758,0.8255,0.841679,0.859253,0.874672
Панельные (6+ этажей),0.0422566,0.0422566,0.0422566,0.0422566,0.0422566,0.0422566,0.0422566,0.0488092,0.0488092,0.0488092,...,0.0488092,0.0488092,0.0488092,0.049823,0.0514577,0.0514577,0.0531727,0.0742663,0.0742663,0.0754501
Монолитные,0.000728864,0.000728864,0.000728864,0.000728864,0.000728864,0.000776482,0.000776482,0.000776482,0.000776482,0.000776482,...,0.00102458,0.00102458,0.00102458,0.00102458,0.00137818,0.00137818,0.00137818,0.00137818,0.00137818,0.00137818
Аварийные,25.2654,25.2654,25.2654,25.2654,25.2654,25.2654,25.2654,25.2654,25.2654,25.2654,...,25.2654,25.2654,25.2654,25.2654,25.2654,25.2654,25.2654,25.2654,25.2654,25.2654


In [41]:
# Сохранение результатов и условий для использования в блокноте сравнения с результатами расчетов на экселевских данных
%store fdf
%store Cond_df
%store HouseSqTot

Stored 'fdf' (DataFrame)
Stored 'Cond_df' (DataFrame)
Stored 'HouseSqTot' (float)


## Cохранение в БД


In [52]:
conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strHousesDBPath))
tfdf=fdf.T
tfdf.to_sql(strHouseDisps_table, if_exists='replace', con=conWork)

In [62]:
# сделать возможность сохранеиня в CSV